In [1]:
# DM Algorithm.
# Matrices can be sorted in ascending or decreasing order according to their diagonals. 
# The current code is set to ascending order and first tie-case option. The user can change these two options in the code below!
# Then the program solves the 3-index Assignment problem. 

import sys
import numpy as np
from pynput import keyboard
import pandas as pd
import random
import math
from statistics import mean, median, pstdev
import time
from scipy.optimize import linear_sum_assignment

#print("Matrices are sorted according to their Diagonals in Descending order.")

start_time = time.time()
Total = 0.00    
num_list = []


def load_data(file_name, ascending_order=True):
    a = np.loadtxt(file_name)
    a = list(a)
    a = [int(i) for i in a]

    b = len(a)
    n = math.pow(b, 1./3.)
    if type(n) != "int":
        n = math.ceil(n)

    for x in a:
        x = int(x)
        num_list.append(x)

    factory =[]
    m = n*n
    for j in range(n):
        lb=j*m
        ub=(j+1)*m 
        factory = factory + num_list[lb:ub]

    factories = [factory[j:j+m] for j in range(0, len(factory), m)]
    
    data = []

    for x in factories:
        mt = np.asarray(x).reshape(n,n)
        mt = np.array(mt)
        #print()
        #print("mt = ",mt)
        mtrace = sum(mt[i][i] for i in range(n))
        mantitrace = sum(mt[i][n-i-1] for i in range(n))
        #print(str(mtrace)+" and "+str(mantitrace))
        maxdiag = max(mtrace, mantitrace)
        #print("Maximum = ",maxdiag)

        data.append([x, maxdiag])

    df = pd.DataFrame(data,columns=['List', 'Diagonals'])
    #print(df, '\n')  

    #print("Rearranging these matrices.")
    df1 = df.sort_values(by=['Diagonals'], ascending=ascending_order)
    #print(df1)

    return (a, factories, df, df1)

# Update the path to load your date=a sample
data_file = r"C:\Users\mehbalim\Desktop\Temp Sep 2021\B&S_3IAP_n10_instance1.txt"

# Make sure you set ascending_order to either True or False.

(a, factories, df, df1) = load_data(data_file, ascending_order = True)
n = len(factories)
   
newList = []
for y in df1.iterrows():
    newList.append(y[1]['List'])
#print()

factories = newList
n=len(factories)

assign = {'Jobs':[],'Machines':[],'Factories':[],'Cost':[]}
df = pd.DataFrame(assign)
df = df.astype('int64', copy=False)
df["Cost"] = df['Cost'].astype('float')

class CustomError(Exception):
    pass

tables = factories
#print()
#print("The 3-Index Assignment Problem is of size", n, ", i.e.", n, "jobs for", n, "machines, in", n, "factories. \n")

def select_One_minimum(matrix, k, option = 0):
    """
    This function selects one min candidate from a list of candidates, i.e. tie-case.
    Set option to select either random tie-case, or firt tie-case or the last.
    For random option = 1, for first or last option = 0 or -1 respectively.
    """
    row_ind, col_ind = linear_sum_assignment(matrix)
    cost_list = matrix[(row_ind, col_ind)]

    #print("Assignment of minimum cost for this matrix is ") 
    #print(cost_list, " The value of the minimum cost is ", min(cost_list), '.\n')
    
    minlyst=[]
    for i in range(k):
        if cost_list[i] == min(cost_list):
            minlyst.append([row_ind[i], col_ind[i]])
    #print(minlyst, '\n')

    min_indices = [i for i in range(k) if cost_list[i] == min(cost_list)]
    
    ## random 
    if option == 1: 
        mi = len(min_indices)
        
        t=(random.randint(0,mi-1))  
        row_ix=min_indices[t]
        
    # first or last (i.e. 0 or -1)     
    else:      
        row_ix=min_indices[option]
                
    global cost_value
    cost_value=min(cost_list)
    
    for j in range(n):
        if j==row_ix:
            global row_indx, col_indx
            row_indx=row_ind[j]
            col_indx=col_ind[j]
            #print("The coordinates of the minimum selected are : index_row =", row_indx, "and index_col =", col_indx,'\n')
            break  
            
def two_matrices(mat):
    fact0=mat[0]
    fact1=mat[1]
    lst=[]
    #print("The two matrices are :", '\n')
    #print("Fact1 = ", fact0, '\n')
    #print("Fact2 = ", fact1, '\n')
    
    for i in range(4):
        v=fact0[i]+fact1[3-i]
        lst.append(v)
        
    #print("List of candidate costs = ", lst,'\n')
    global opt_cost, cost0 , cost1
    opt_cost = min(lst) 
    #print("Optimal cost = ", opt_cost,'\n')
    #print("Index of Optimal cost = ", lst.index(opt_cost), '\n')
    
    global ix, indx, jndx, kndx
    ix=lst.index(opt_cost)
    
    cost0=fact0[ix] 
    cost1=fact1[3-ix]
                           
    if ix==0:
        indx=0
        jndx=0
    
    elif ix==1:
        indx=0
        jndx=1

    elif ix==2:
        indx=1
        jndx=0
 
    else:
        indx=1
        jndx=1
    kndx=n-2  
     
    #print("Optimal cost = ", cost0, " + ", cost1, " = ", opt_cost, '.\n') 
   
    global assign0, assign1
    assign0={'Jobs':indx, 'Machines':jndx,'Factories':kndx,'Cost':cost0}
    assign1={'Jobs':1-indx, 'Machines':1-jndx,'Factories':kndx+1,'Cost':cost1}

k = n
factnb=-1
while k!=2:
    newT = []
    i = 0
    #print("The value of k=", k, '\n')
    for t in tables:
        if i == 0:
            arr=np.asarray(t).reshape(k,k)
            arr = np.array(arr)
            #print(arr, '\n')
            
            """Select which tie-case to examine, 0, -1 or 1 for first, last or random."""
            
            select_One_minimum(arr, k, option = 0)
            factnb=factnb+1
            
            assign={'Jobs':row_indx, 'Machines':col_indx,'Factories':factnb,'Cost':cost_value}
            df=df.append(assign, ignore_index=True)
            df = df.astype('int64', copy=False)
            Total=Total + cost_value
            new_table=[]
            ix=row_indx
            iy=col_indx
        else:
            #print("Reduce the matrice ",i)
            arr=np.asarray(t).reshape(k,k)
            #print(arr, '\n')
            row_idx = ix
            col_idx = iy
            new_mat = np.delete(arr, row_idx, 0)
            new_mat = np.delete(new_mat, col_idx, 1)
            #print(new_mat,'\n')
            new_mat = np.array(new_mat).flatten().tolist()
            newT.append(new_mat)
        i=i+1
    k=k-1
    tables=newT
    #print(k,"matrices", tables, '\n')
    
two_matrices(tables)
Total=Total + opt_cost
df = df.append(assign0, ignore_index=True)
df = df.append(assign1, ignore_index=True)
df = df.astype('int64', copy=False)
      
#print(df,'\n')

print("Total cost = %.2f" % Total, " -- Running Time -- %.8f seconds --" % (time.time() - start_time), '\n')


Total cost = 129.00  -- Running Time -- 0.00843835 seconds -- 

